In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
#%cd drive/My Drive/Intern/Image_Seg

In [3]:
#%cd unet

In [4]:
from model import *

Using TensorFlow backend.


In [5]:
seed = 1

In [6]:
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')

In [7]:
def gen(train_path ,image_folder = 'image', mask_folder = 'label',image_color_mode = "grayscale",
        mask_color_mode = "grayscale", target_size = (256,256), flag_multi_class = False, batch_size = 2,data_gen_args=data_gen_args
       ):
    image_datagen = ImageDataGenerator(**data_gen_args)
    mask_datagen = ImageDataGenerator(**data_gen_args)
    image_generator = image_datagen.flow_from_directory(
        train_path,
        classes = [image_folder],
        class_mode = None,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        seed = seed)
    mask_generator = mask_datagen.flow_from_directory(
        train_path,
        classes = [mask_folder],
        class_mode = None,
        color_mode = mask_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        seed = seed)
    train_generator = zip(image_generator, mask_generator)
    
    for (img,mask) in train_generator:
        print("Loading gene")
        if(np.max(img) > 1):
            img = img / 255
            mask = mask /255
            mask[mask > 0.5] = 1
            mask[mask <= 0.5] = 0
            yield(img, mask)

#     return train_generator

In [8]:
import medicalai as ai
datasetFolderPath = 'data/membrane/'
(IMG_HEIGHT,IMG_WIDTH) = (256,256)
augment = ai.AUGMENTATION(rotation_range = 12, 
                          fill_mode='nearest', 
                          width_shift_range=0.1, 
                          height_shift_range=0.1, 
                          brightness_range = (0.9, 1.1), 
                          zoom_range=(0.85, 1.15), 
                          rescale= 1./255,)
dsHandler = ai.segmentaionGenerator(folder=datasetFolderPath,targetDim=(IMG_HEIGHT,IMG_WIDTH), 
                                        augmentation=augment, class_mode=None,
                                        batch_size = 1,color_mode="grayscale",
                                        image_folder_name = "image", mask_folder_name = "label")

loading image gen
Found 30 images belonging to 1 classes.
loading mask gen
Found 30 images belonging to 1 classes.
adjusting image


In [9]:
model = unet()
model_checkpoint = ModelCheckpoint('unet_membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)

In [10]:

#myGene = gen(train_path = 'data/membrane/train')

#myGene = dsHandler.load_generator()

myGene = ai.segmentGen(train_path = 'data/membrane/train').load_gen()
model.fit( myGene,steps_per_epoch=1,epochs=10,callbacks=[model_checkpoint])

Found 30 images belonging to 1 classes.
Found 30 images belonging to 1 classes.
Loading gene
Epoch 1/10
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Loading gene
1/1 [==============================] - ETA: 0s - loss: 0.7536 - accuracy: 0.1949
Epoch 00001: loss improved from inf to 0.75356, saving model to unet_membrane.hdf5
1/1 [==============================] - 2s 2s/step - loss: 0.7536 - accuracy: 0.1949
Epoch 2/10
Lo

In [11]:
testGene = testGenerator("data/membrane/test")
model = unet()
model.load_weights("unet_membrane.hdf5")
results = model.predict_generator(testGene,30,verbose=1)
saveResult("results",results)

NameError: name 'testGenerator' is not defined